In [1]:
import os
from haystack.document_stores import FAISSDocumentStore
from haystack.pipelines.standard_pipelines import TextIndexingPipeline
from haystack.nodes import CsvTextConverter

doc_dir = "data/raw"

document_store = FAISSDocumentStore(similarity="dot_product")

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
#indexing_pipeline.run_batch(file_paths=list(filter(lambda x: x.endswith('.txt'), files_to_index)))

indexing_pipeline = TextIndexingPipeline(document_store, text_converter=CsvTextConverter())
indexing_pipeline.run_batch(file_paths=list(filter(lambda x: x.endswith('.csv'), files_to_index)))

c:\Users\d103771\Documents\GitHub\SAR3\.venv-hey\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.
Preprocessing: 100%|██████████| 31/31 [00:00<00:00, 481.67docs/s]
Writing Documents: 10000it [00:00, 40402.03it/s]         


{'documents': [<Document: {'content': 'Dove posso lavorare in modalità agile?', 'content_type': 'text', 'score': None, 'meta': {'answer': 'Ovunque tu voglia, compatibilmente con la tua sicurezza personale e quella dei dati e delle informazioni trattate. Non hai alcun obbligo di concordare o comunicare il luogo di lavoro con il tuo responsabile.', '_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '249f61604d04661cc72e1a6a2c48238f'}>,
  <Document: {'content': 'Posso lavorare in modalità agile da un Paese estero?', 'content_type': 'text', 'score': None, 'meta': {'answer': 'sì è possibile, compatibilmente con la necessità di lavorare in presenza in alcune giornate.', '_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '7edf516046590783ab658300ef415a0d'}>,
  <Document: {'content': 'Quanto dura la giornata di lavoro agile?', 'content_type': 'text', 'score': None, 'meta': {'answer': 'Tendenzialmente dura quanto una qualunque giornata di lavoro in coeren

In [2]:
from haystack.nodes import DensePassageRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="voidful/dpr-question_encoder-bert-base-multilingual",
    passage_embedding_model="voidful/dpr-ctx_encoder-bert-base-multilingual"
)

document_store.update_embeddings(retriever)

reader = FARMReader("anakin87/electra-italian-xxl-cased-squad-it")

pipe = ExtractiveQAPipeline(reader, retriever)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
Documents Processed: 10000 docs [00:21, 468.10 docs/s]        


In [4]:
prediction = pipe.run(
    query="Posso lavorare dall'estero?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.72s/ Batches]


Query: Posso lavorare dall'estero?
Answers:
[   {   'answer': 'mezza giornata in presenza e mezza giornata in remoto',
        'context': 'Posso lavorare mezza giornata in presenza e mezza giornata '
                   'in remoto?'},
    {   'answer': '16',
        'context': 'Posso iniziare a lavorare alle 10 o finire di lavorare '
                   'alle 16?'},
    {   'answer': 'Una giornata di formazione fuori sede',
        'context': 'Una giornata di formazione fuori sede è considerata nel '
                   'conteggio dei giorni in presenza?'},
    {   'answer': 'in modalità agile',
        'context': 'Posso lavorare in modalità agile da un Paese estero?'},
    {   'answer': 'Confronto col responsabile (che in pratica è solo una '
                  'comunicazione della volontà di accedere all’orario ridotto '
                  'su base volontaria) e successiva richiesta del part time '
                  'tramite apposito modulo o mail',
        'context': 'Confronto col resp

In [5]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query = "Posso lavorare dalla Francia?"
result = pipeline.run(query, params={"Retriever": {"top_k": 5}})
result

{'documents': [<Document: {'content': 'Posso iniziare a lavorare alle 10 o finire di lavorare alle 16?', 'content_type': 'text', 'score': 0.5089208070167367, 'meta': {'answer': 'sì, compatibilmente con l’organizzazione del lavoro, puoi scegliere liberamente come distribuire le ore di lavoro tra le 07.00 e le 19.00.', '_split_id': 0, 'vector_id': '25'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'd6bbf418bcd4f92b769849334e03df1f'}>,
  <Document: {'content': 'La prestazione è disciplinata da un accordo scritto tra azienda e dipendente, il cui modello è riportato nell’allegato A) del presente documento. La durata della stessa è concordata con il proprio responsabile – tenute presenti le esigenze organizzative – per un periodo non eccedente i 12 mesi, rinnovabile a scadenza. Per proprio responsabile sì fa riferimento ai dirigenti di settore o al Funzionario diretto?', 'content_type': 'text', 'score': 0.5073108129494602, 'meta': {'answer': 'Al responsabile diretto.', '_split_id':

In [13]:
from haystack.nodes import Seq2SeqGenerator
from haystack.pipelines import GenerativeQAPipeline

generator = Seq2SeqGenerator(model_name_or_path="efederici/bart-lfqa-it")
pipe = GenerativeQAPipeline(generator, retriever)

prediction = pipe.run(
    query="Posso lavorare dalla Francia?",
    params={
        "Retriever": {"top_k": 10}
    }
)

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Exception: Exception while running node 'Generator': "Seq2SeqGenerator doesn't have input converter registered for efederici/bart-lfqa-it. Provide custom converter for efederici/bart-lfqa-it in Seq2SeqGenerator initialization"
Enable debug logging to see the data that was passed when the pipeline failed.